## Carga de datos

In [1]:
#Importamos las librerias

import os  # Importamos el módulo os para interactuar con el sistema operativo, como acceder a variables de entorno y rutas de archivos.
import requests  # Importamos el módulo requests para realizar solicitudes HTTP.
import pandas as pd  # Importamos pandas bajo el alias pd para manipulación y análisis de datos.
from sklearn.feature_selection import SelectKBest  # Importamos la clase SelectKBest de scikit-learn para la selección de características.
from sklearn.feature_selection import f_classif  # Importamos f_classif de scikit-learn, que es una función de puntuación ANOVA para clasificación.
import sqlite3  # Importamos el módulo sqlite3 para trabajar con bases de datos SQLite.


In [2]:
# Se procede a comenta, porque la ruta data/covertype/covertype_train.csv fue instanciada con dvc pull

'''
## Descargamos el conjunto de datos

# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)
'''

"\n## Descargamos el conjunto de datos\n\n# Directory of the raw data files\n_data_root = './data/covertype'\n# Path to the raw training data\n_data_filepath = os.path.join(_data_root, 'covertype_train.csv')\n# Download data\nos.makedirs(_data_root, exist_ok=True)\nif not os.path.isfile(_data_filepath):\n    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/\n    url = 'https://docs.google.com/uc?export=     download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'\n    r = requests.get(url, allow_redirects=True, stream=True)\n    open(_data_filepath, 'wb').write(r.content)\n"

## Lectura de datos

In [3]:
# Cargamos los datos desde un archivo CSV utilizando pandas y lo almacenamos en el DataFrame 'data'.
data = pd.read_csv("data/covertype/covertype_train.csv")

# Mostramos las primeras filas del DataFrame para verificar la correcta carga de los datos.
data.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [4]:
# Utilizamos el método info() para obtener información sobre el DataFrame 'data',
# incluyendo el número de filas y columnas, los tipos de datos de cada columna,
# y la cantidad de valores no nulos en cada columna.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

## Selección de Características

In [5]:
# Creamos la variable X que contiene todas las características (predictoras) eliminando la columna 'Cover_Type'.
X = data.drop(columns='Cover_Type')

# Creamos la variable y que contiene la columna 'Cover_Type', que es nuestra variable objetivo.
y = data['Cover_Type']

# Identificamos los tipos de columnas presentes en el DataFrame X.
# Las columnas que no son de tipo 'int64' se consideran categorías, mientras que las que son de tipo 'int64' se consideran numéricas.
colum_cat = X.select_dtypes(exclude = ['int64']).columns
colum_num = X.select_dtypes(include = ['int64']).columns

# Creamos dos nuevos DataFrames, uno que contiene solo las características numéricas y otro que contiene solo las características categóricas.
datos_num = X[colum_num]
datos_cat = X[colum_cat]

In [6]:
datos_num.shape

(116203, 10)

In [7]:
datos_cat.shape

(116203, 2)

In [8]:
def feature_selection(df, y, k):
    """
    Función para realizar la selección de características utilizando SelectKBest.

    Parámetros:
    - df: DataFrame que contiene las características.
    - y: Serie que contiene la variable objetivo.
    - k: Número de características a seleccionar.

    Retorna:
    - table: DataFrame que muestra las características seleccionadas y si se retienen o no.
    - df_fs: DataFrame que contiene las características seleccionadas.
    """

    # Seleccionamos las mejores características utilizando SelectKBest y el método f_classif.
    kbest = SelectKBest(f_classif, k=k)
    X_new = kbest.fit_transform(df, y)
    
    # Creamos una tabla que muestra las características originales y si se retienen o no.
    table = pd.DataFrame({
        "Columna": df.columns,
        "Retener": kbest.get_support()
    })
    
    # Creamos un nuevo DataFrame que contiene solo las características seleccionadas.
    df_fs = df[df.columns[kbest.get_support(indices=True)]]
    
    return table, df_fs

# Llamamos a la función feature_selection con datos_num, y y 8 como parámetros.
table, df_fs = feature_selection(datos_num, y, 8)


In [9]:
table, df_fs = feature_selection(datos_num, y, 8)
table

,Columna,Retener
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


In [10]:
df_fs.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,7,67,11,1015,233,234,1570
1,2876,18,485,71,2495,192,202,1557
2,3171,2,277,9,4374,213,237,1052
3,3087,13,190,31,4774,193,221,752
4,2835,10,212,41,3596,231,242,3280


## Almacenamiento en base de datos

In [11]:
# Concatenamos los DataFrames df_fs (contiene las características seleccionadas), datos_cat (contiene las características categóricas) y y (la variable objetivo).
# Lo hacemos a lo largo del eje 1 para concatenar por columnas.
df_final = pd.concat([df_fs, datos_cat, y], axis=1)


In [12]:
df_final.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,11,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,71,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,9,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,31,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,41,3596,231,242,3280,Rawah,C4744,1


In [13]:
df_final.shape

(116203, 11)

In [14]:
# Se procede a comenta, porque la ruta data/data_mod fue instanciada con dvc pull

'''
# Directorio donde guardar los datos
directoriodatos = "data/data_mod"

# Comprobar si el directorio existe, si no, crearlo
if not os.path.exists(directoriodatos):
    os.makedirs(directoriodatos)

# Guardar el DataFrame en formato CSV en el directorio especificado
df_final.to_csv(os.path.join(directoriodatos, "datos_finales.csv"), index=False)
'''

'\n# Directorio donde guardar los datos\ndirectoriodatos = "data/data_mod"\n\n# Comprobar si el directorio existe, si no, crearlo\nif not os.path.exists(directoriodatos):\n    os.makedirs(directoriodatos)\n\n# Guardar el DataFrame en formato CSV en el directorio especificado\ndf_final.to_csv(os.path.join(directoriodatos, "datos_finales.csv"), index=False)\n'